In [1]:
import requests
from requests.auth import HTTPProxyAuth
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re
import socks
import socket
import sqlite3
from stem import Signal
from stem.control import Controller 
import numpy as np
import math

In [ ]:
## Steps for Git
Git add <file> 
Git commit -m "comments" 
Git push origin master

In [9]:
## Initizlization

#Create SQLite
conn = sqlite3.connect('songlyrics.db')
# df = pd.read_sql_query("select * from lyrics_url;",conn)

In [6]:
session = requests.session()
session.proxies = {'https':'socks5://127.0.0.1:9050','http':'socks5://127.0.0.1:9050'}

## Get Christian Artist List

In [40]:
artist_list = pd.read_csv('christian_artist.csv')

In [22]:
list_url = 
['http://www.songlyrics.com/twila-paris-lyrics/',
 'http://www.songlyrics.com/wayne-watson-lyrics/'
]

'don moen'

In [75]:
def clean_artist(x):
#     artist_list[artist_list['Genre']=='Band'].iloc[20,0].split('\xa0')[0]
    return x['Artist'].split('\xa0')[0]

artist_list['clean_artist'] = artist_list.apply(lambda x: clean_artist(x),axis=1)

In [77]:
def convert_url(x):
    return "http://www.songlyrics.com/"+x['clean_artist'].lower().replace(',','').replace('.','').replace(' &','').replace(' ','-')+"-lyrics/"

artist_list['url'] = artist_list.apply(lambda x: convert_url(x),axis=1)

In [86]:
artist_list[artist_list.duplicated(subset=['clean_artist'])]
final_list = artist_list.drop_duplicates(subset=['clean_artist'],inplace=True)
# artist_list[artist_list['clean_artist']=='Newsboys']
artist_list.to_csv('christian_artist_clean.csv')

In [99]:
# 0 not yet
# 1 downloaded
# 2 artist name don't match
artist_list['status'] = 0

In [118]:
url_list = artist_list[artist_list['Genre']=='Worship'].iloc[11:15,:]

In [119]:
url_list

,Artist,Genre,url,clean_artist,status
991,Robin Mark,Worship,http://www.songlyrics.com/robin-mark-lyrics/,Robin Mark,0
992,Matt Price,Worship,http://www.songlyrics.com/matt-price-lyrics/,Matt Price,0
993,Sonicflood,Worship,http://www.songlyrics.com/sonicflood-lyrics/,Sonicflood,0
994,Kim Walker,Worship,http://www.songlyrics.com/kim-walker-lyrics/,Kim Walker,0


In [120]:
user_agent_list = ['Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1623.0 Safari/537.36',
                   'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 7.0; InfoPath.3; .NET CLR 3.1.40767; Trident/6.0; en-IN)',
                   'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
                   'Mozilla/5.0 (X11; Ubuntu; Linux armv7l; rv:17.0) Gecko/20100101 Firefox/17.0'
                  ]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

base_url = "http://www.songlyrics.com/steve-green-lyrics/"

columns = ['url','title','status','artist']
dflyrics = pd.DataFrame(columns=columns)

for key,row in url_list.iterrows():
    # response = session.get(url, headers=headers)
    pause = randint(3,5)
    time.sleep(pause)
    base_url = row['url']
    response = requests.get(base_url,headers=headers)
    response_url = ' '.join(response.url.split("/")[-2].split("-")[0:-1])
    artist = base_url.split("/")[-2].replace('lyrics','').replace('-',' ').strip().lower()
    
    
    if response_url != artist:
        url_list.loc[key,'status'] = 2
    else:
        url_list.loc[key,'status'] = 1
        data = response.text
        song_url = BeautifulSoup(data,"lxml")
        song_url_list = song_url.findAll('table',attrs={'class','tracklist'})
        for s in song_url_list[0].findAll('a'):
            url = s.get('href')
            title = s.get_text()
            status = 0 
        #     artist = base_url.split("/")[-2].replace('lyrics','').replace('-',' ').strip()
            row = [url,title,status,artist]
            dflyrics.loc[len(dflyrics)] = row  

dflyrics.to_sql('songlyrics_url',conn,if_exists='append')

In [121]:
url_list

,Artist,Genre,url,clean_artist,status
991,Robin Mark,Worship,http://www.songlyrics.com/robin-mark-lyrics/,Robin Mark,1
992,Matt Price,Worship,http://www.songlyrics.com/matt-price-lyrics/,Matt Price,1
993,Sonicflood,Worship,http://www.songlyrics.com/sonicflood-lyrics/,Sonicflood,1
994,Kim Walker,Worship,http://www.songlyrics.com/kim-walker-lyrics/,Kim Walker,1


In [122]:
songlyrics = pd.read_sql_query('select * from songlyrics_url',conn)
len(songlyrics)

1524

In [123]:
songlyrics

,index,url,title,status,artist
0,0,http://www.songlyrics.com/don-moen/the-greatne...,The Greatness Of You,0.0,don moen
1,1,http://www.songlyrics.com/don-moen/o-god-of-ab...,O God Of Abraham,0.0,don moen
2,2,http://www.songlyrics.com/don-moen/grace-is-en...,Grace Is Enough,0.0,don moen
3,3,http://www.songlyrics.com/don-moen/saviour-kin...,Saviour King,0.0,don moen
4,4,http://www.songlyrics.com/don-moen/thank-you-l...,Thank You Lord,0.0,don moen
5,5,http://www.songlyrics.com/don-moen/i-will-sing...,I Will Sing,0.0,don moen
6,6,http://www.songlyrics.com/don-moen/i-ll-say-ye...,I'll Say Yes,0.0,don moen
7,7,http://www.songlyrics.com/don-moen/thank-god-i...,Thank God I'm Free,0.0,don moen
8,8,http://www.songlyrics.com/don-moen/our-father-...,Our Father,0.0,don moen
9,9,http://www.songlyrics.com/don-moen/arise-lyrics/,Arise,0.0,don moen


In [58]:

dflyrics.reset_index(inplace=True)
dflyrics['key'] = dflyrics['index']
del dflyrics['index']
#alter table songlyrics_url add column fullhtml text;

dflyrics.to_sql('songlyrics_url',conn,if_exists='replace')

In [ ]:
def set_new_ip():
    """Change IP using TOR"""
    with Controller.from_port(port=9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)

cursor = conn.cursor()
        
batch = 1000
mini_batch = 100 
lower_limit = 3
upper_limit = 5

iterations = 0
count_query = pd.read_sql_query("select count(*) as cnt from songlyrics_url where status = 0;",conn)
count = int(count_query['cnt'][0])
# initial_count = count - batch
initial_count = 1700 

while count > initial_count : 

    count_query = pd.read_sql_query("select count(*) as cnt from songlyrics_url where status = 0;",conn)
    count = int(count_query['cnt'][0])

    # Randomize 
    set_new_ip()
    rand_user_agent = randint(0,len(user_agent_list)-1)
    headers['User-Agent'] = user_agent_list[rand_user_agent]
    elapsed_time = 0 
    # Scrap
    scrap = pd.read_sql_query("select key,url from songlyrics_url where status = 0 limit "+str(mini_batch)+";",conn)
    t0 = time.time()
    for key,row in scrap.iterrows():

        pause = randint(lower_limit,upper_limit)
        time.sleep(pause)

        key = row['key']
        url = row['url']
        response = session.get(url, headers=headers)
        elapsed_time += response.elapsed.total_seconds()
        data = response.text

        
            
        cursor.execute("""UPDATE songlyrics_url SET fullhtml = ?, status = ? WHERE key = ? """,
              (data,1,key))
        conn.commit()
            
    t1 = time.time()
    print("Interations :",iterations," Per Request:" ,elapsed_time/mini_batch," Total:",t1-t0)
    # print("Initial_count ",initial_count," Current:",count)
    iterations = iterations + 1
    

cursor.close()
conn.close()

Interations : 0  Per Request: 0.9110592000000001  Total: 583.5309026241302


In [ ]:
# Maranatha Music / Integrity Music / Vineyard Music / Passion

In [5]:
#Page Extract

base_url = "http://www.songlyrics.com/maranatha!-music/healer-lyrics/"
response = requests.get(base_url)
data = response.text
soup = BeautifulSoup(data,"lxml")


In [7]:
lyrics = soup.find('p',attrs={'id':'songLyricsDiv'})
lyrics.get_text()

"*applauses*\n\nInstrumental\n\nYou hold my every moment\nYou calm my raging seas\nYou walk with me through fire\nAnd heal all my disease\n\nI trust in You\nI trust in You\n\nI believe You're my Healer\nI believe You are all I need,\nOh\n\nI believe You're my portion\nI believe You're more than enough for me\nJesus You're all I need\n\nYou hold my every moment\nYou calm my raging seas\nYou walk with me through fire\nAnd heal all my disease\n\nI trust in You\nI trust in You\n\nI believe You're my Healer\nI believe You are all I need,\nOh\n\nAnd I believe You're my portion\nI believe You're more than enough for me\nJesus You're all I need\n\nNothing is impossible for You\nNothing is impossible\nNothing is impossible for You\nYou hold my world in Your hands\n\nNothing is impossible for You\nNothing is impossible\nNothing is impossible for You\nYou hold my world in Your hands\nOh\n\nI believe You're my healer\nI believe You are all I need\n\nI believe You're my portion\nI believe You're mo